In [1]:
import numpy as np
import matplotlib.pyplot as plt
from celmech.disturbing_function import df_coefficient_Ctilde
from celmech.disturbing_function import evaluate_df_coefficient_dict
from celmech.disturbing_function import laplace_b as b

In [2]:
def b1(alpha):
    return b(1.5, 1, 0, alpha)
def b2(alpha):
    return b(1.5, 2, 0, alpha)

In [3]:
def Mprime(alpha, mass):
    m1, m2, m3 = mass
    m_tot = m1+m2+m3
    alpha12, alpha23 = alpha
    alpha13 = alpha12+alpha23
    
    R = np.array([[np.sqrt(m1/m_tot), np.sqrt(m2/m_tot), np.sqrt(m3/m_tot)],
             [-np.sqrt(m3/(m1+m3)), 0, np.sqrt(m1/(m1+m3))],
             [np.sqrt(m1*m2/(m1+m3)/m_tot), -np.sqrt((m1+m3)/m_tot), np.sqrt(m2*m3/(m1+m3)/m_tot)]])
       
    M = np.array([[b1(alpha12)*m2+b1(alpha13)*m3, -b2(alpha12)*np.sqrt(m1*m2), -b2(alpha13)*np.sqrt(m1*m3)],
             [-b2(alpha12)*np.sqrt(m1*m2), b1(alpha12)*m1+ b1(alpha23)*m3, -b2(alpha23)*np.sqrt(m2*m3)],
             [-b2(alpha13)*np.sqrt(m1*m3), -b2(alpha23)*np.sqrt(m2*m3), b1(alpha13)*m1+b1(alpha23)*m2]])*(-2*alpha[2])*np.pi/4
    
    return np.matmul(np.matmul(R, M), np.transpose(R))

In [4]:
Mprime((0.9, 0.91),(1e-8,1e-8,1e-8))

array([[-4.63070965e-08, -3.85151626e-10,  4.26369912e-09],
       [-3.85151626e-10, -1.03631642e-06, -1.50676113e-07],
       [ 4.26369912e-09, -1.50676113e-07, -2.50139292e-06]])

$$ R \cdot \begin{pmatrix}\vec{G_1}\\ \vec{G_2}\\ \vec{G_3} \end{pmatrix} = \begin{pmatrix}\vec{S_1}\\ \vec{S_2}\\ \vec{S_3} \end{pmatrix} $$

In [5]:
def StoG(mass, amp, phi):
    m1, m2, m3 = mass
    m_tot = m1+m2+m3
    A1, A2, A3 = amp
    p1, p2, p3 = phi
    
    e1_x = 1/np.sqrt(m1)*(np.sqrt(m1/m_tot)*A1*np.cos(p1) - np.sqrt(m3/(m1+m3))*A2*np.cos(p2) + np.sqrt(m1*m2/(m_tot*(m1+m3)))*A3*np.cos(p3))
    e2_x = 1/np.sqrt(m2)*(np.sqrt(m2/m_tot)*A1*np.cos(p1) - np.sqrt((m1+m3)/m_tot)*A3*np.cos(p3))
    e3_x = 1/np.sqrt(m3)*(np.sqrt(m3/m_tot)*A1*np.cos(p1) + np.sqrt(m1/(m1+m3))*A2*np.cos(p2) + np.sqrt(m2*m3/(m_tot*(m1+m3)))*A3*np.cos(p3))
    
    e1_y = 1/np.sqrt(m1)*(np.sqrt(m1/m_tot)*A1*np.sin(p1) - np.sqrt(m3/(m1+m3))*A2*np.sin(p2) + np.sqrt(m1*m2/(m_tot*(m1+m3)))*A3*np.sin(p3))
    e2_y = 1/np.sqrt(m2)*(np.sqrt(m2/m_tot)*A1*np.sin(p1) - np.sqrt((m1+m3)/m_tot)*A3*np.sin(p3))
    e3_y = 1/np.sqrt(m3)*(np.sqrt(m3/m_tot)*A1*np.sin(p1) + np.sqrt(m1/(m1+m3))*A2*np.sin(p2) + np.sqrt(m2*m3/(m_tot*(m1+m3)))*A3*np.sin(p3))
    
    e1, e2, e3 = np.sqrt(e1_x**2 + e1_y**2), np.sqrt(e2_x**2 + e2_y**2), np.sqrt(e3_x**2 + e3_y**2)
    pom1, pom2, pom3 = np.arctan2(e1_y,e1_x), np.arctan2(e2_y,e2_x), np.arctan2(e3_y,e3_x)
    
    return (e1, e2, e3), (pom1, pom2, pom3)

In [6]:
def GtoS(alpha, mass, ecc, pomega):
    alpha12, alpha23, alpha13 = alpha
    m1, m2, m3 = mass
    m_tot = m1+m2+m3
    ecc1, ecc2, ecc3 = ecc
    pom1, pom2, pom3 = pomega

    e1_x, e2_x, e3_x = ecc1*np.cos(pom1), ecc2*np.cos(pom2), ecc3*np.cos(pom3)
    e1_y, e2_y, e3_y = ecc1*np.sin(pom1), ecc2*np.sin(pom2), ecc3*np.sin(pom3)
    
    amp1 = np.sqrt(1/m_tot) * np.sqrt((m1*e1_y + m2*e2_y + m3*e3_y)**2 + (m1*e1_x + m2*e2_x + m3*e3_x)**2)
    amp2 = np.sqrt(m1*m3/(m1+m3)) * np.sqrt((e3_y-e1_y)**2 + (e3_x-e1_x)**2)
    amp3 = np.sqrt(m2/((m1+m3)*m_tot)) * np.sqrt((m3*(e3_y-e2_y)-m1*(e2_y-e1_y))**2+(m3*(e3_x-e2_x)-m1*(e2_x-e1_x))**2)
    
    phase1 = np.arctan2(m1*e1_y+m2*e2_y+m3*e3_y, m1*e1_x+m2*e2_x+m3*e3_x)
    phase2 = np.arctan2(e3_y-e1_y, e3_x-e1_x)
    phase3 = np.arctan2(m3*(e3_y-e2_y)-m1*(e2_y-e1_y), m3*(e3_x-e2_x)-m1*(e2_x-e1_x))
    
    omega2 = 1/(m1+m3)*(m2*m3*b1(alpha12) + m1*m2*b1(alpha23) + (m1**2+m3**2)*b1(alpha13) + 2*m1*m3*b2(alpha13)) * (2*alpha13)*np.pi/4
    omega3 = (((m1+m3)**2+m2**2)/((m1+m3)*m_tot)*(m1*b1(alpha12) + m3*b1(alpha23)) + 2*m2/m_tot*(m1*b2(alpha12)+m3*b2(alpha23))) * (2*alpha13)*np.pi/4
    
    return amp1, amp2, amp3, phase1, phase2, phase3, omega2, omega3

In [7]:
def breathingON(mass, A3):
    m1, m2, m3 = mass
    m_tot = m1+m2+m3
    
    A3_b = A3 / np.sqrt(2)
    A2_b = (np.sqrt((m1+m3)/(m2*m_tot)) + np.sqrt(m2/((m1+m3)*m_tot)))*A3_b / np.sqrt(m3/(m1*(m1+m3)))
    return A2_b, A3_b

In [ ]:
def plotEcc(alpha, mass, ecc, pomega, time):
    m1, m2, m3 = mass
    m_tot = m1+m2+m3
    A1, A2, A3, p1, p2, p3, w2, w3 = GtoS(alpha, mass, ecc, pomega)
#     print(f"A1={A1},A2={A2},A3={A3}")
#     print(f"w2={w2},w3={w3}")
    
    t = np.linspace(0,time,400)
    
    e1_x = 1/np.sqrt(m1)*(np.sqrt(m1/m_tot)*A1*np.cos(p1) - np.sqrt(m3/(m1+m3))*A2*np.cos(w2*t+p2) + np.sqrt(m1*m2/(m_tot*(m1+m3)))*A3*np.cos(w3*t+p3))
    e2_x = 1/np.sqrt(m2)*(np.sqrt(m2/m_tot)*A1*np.cos(p1) - np.sqrt((m1+m3)/m_tot)*A3*np.cos(w3*t+p3))
    e3_x = 1/np.sqrt(m3)*(np.sqrt(m3/m_tot)*A1*np.cos(p1) + np.sqrt(m1/(m1+m3))*A2*np.cos(w2*t+p2) + np.sqrt(m2*m3/(m_tot*(m1+m3)))*A3*np.cos(w3*t+p3))
    
    e1_y = 1/np.sqrt(m1)*(np.sqrt(m1/m_tot)*A1*np.sin(p1) - np.sqrt(m3/(m1+m3))*A2*np.sin(w2*t+p2) + np.sqrt(m1*m2/(m_tot*(m1+m3)))*A3*np.sin(w3*t+p3))
    e2_y = 1/np.sqrt(m2)*(np.sqrt(m2/m_tot)*A1*np.sin(p1) - np.sqrt((m1+m3)/m_tot)*A3*np.sin(w3*t+p3))
    e3_y = 1/np.sqrt(m3)*(np.sqrt(m3/m_tot)*A1*np.sin(p1) + np.sqrt(m1/(m1+m3))*A2*np.sin(w2*t+p2) + np.sqrt(m2*m3/(m_tot*(m1+m3)))*A3*np.sin(w3*t+p3))
    
    e1 = np.sqrt(e1_x**2 + e1_y**2)
    e2 = np.sqrt(e2_x**2 + e2_y**2)
    e3 = np.sqrt(e3_x**2 + e3_y**2)
    
    e21 = np.sqrt((e2_x-e1_x)**2 + (e2_y-e1_y)**2)
    e32 = np.sqrt((e3_x-e2_x)**2 + (e3_y-e2_y)**2)
    
    fig, axarr = plt.subplots(nrows=5, figsize=(12,14))
    axarr[0].plot(t, e1, 'k.')
    axarr[0].set_ylabel(r"$e_{1}$")
    
    axarr[1].plot(t, e2, 'k.')
    axarr[1].set_ylabel(r"$e_{2}$")
    
    axarr[2].plot(t, e3, 'k.')
    axarr[2].set_ylabel(r"$e_{3}$")
    
    axarr[3].plot(t, e21, 'k.')
    axarr[3].set_ylabel(r"$e_{21}$")
    
    axarr[4].plot(t, e32, 'k.')
    axarr[4].set_ylabel(r"$e_{32}$")
    

In [ ]:
alpha = 0.87, 0.87, 0.87**2
mass = 1e-6,1e-7,1e-7
ecc = 0.01, 0.01, 0.01
pomega = 0, 0, np.pi

amp1, amp2, amp3, phase1, phase2, phase3, omega2, omega3 = GtoS(alpha, mass, ecc, pomega)

A2, A3 = breathingON(mass,amp3)
print(A2, A3)
ecc, pomega = StoG(mass, (amp1, A2, A3), (phase1, phase2, phase3 + np.pi/2))

plotEcc(alpha, mass, ecc, pomega, 5e6)

In [ ]:
ecc, pomega = StoG(mass, (amp1, 0, amp3), (phase1, phase2, phase3))
print(ecc, pomega)
plotEcc(alpha, mass, ecc, pomega,5e6)